# Representação de estruturas químicas

Para criar modelos de QSAR/QSPR e de aprendizado de máquina, precisamos converter as estruturas químicas a um formato que possa ser interpretado pelos algoritmos utilizados. 

Ainda que existam algoritmos capazes de interpretar estruturas moleculares representadas na forma de texto (*Recurrent Neural Networks*, *Transformers*) ou grafos (*Message Passing Neural Networks*, *Graph Neural Networks*), a conversão de estruturas químicas a um formato numérico segue sendo o método mais usado para a criação de modelos preditivos.

## Descritores moleculares

Os descritores moleculares representam de forma numérica alguma característica das estruturas químicas. 

De acordo com Todeschini e Consonni no livro [Molecular Descriptors for Chemoinformatics](https://onlinelibrary.wiley.com/doi/book/10.1002/9783527628766) (en tradução livre):

    "O descritor molecular é o resultado final de um procedimento lógico e matemático que transforma informações químicas codificadas em uma representação simbólica de uma molécula em um número útil ou no resultado de algum experimento padronizado."

Além disso, segundo os autores, um descritor molecular deve cumprir os seguintes requisitos:

1. Invariância com relação à rotulagem e numeração de átomos
2. Invariância com relação à rotação e translação
3. Uma definição computável algoritmicamente inequívoca
4. Valores em um intervalo numérico adequado para o conjunto de moléculas estudadas


Caso tenha alguma familiaridade com quiminformática, pode conhecer os *fingerprints* moleculares, que são descritores moleculares baseados na identificação de subestruturas. Porém, devido a suas particularidades e amplo uso na quiminformática, dedicaremos outros Notebooks para essa classe de descritores. Neste Notebook, vamos usar o termo **"Descritores"** para nos referirmos aos descritores além dos *fingerprints*.

## Importando os módulos necessários

In [1]:
import numpy as np
print("numpy version:", np.__version__)
import pandas as pd
print("numpy version:", pd.__version__)
import rdkit
print("rdkit version:", rdkit.__version__)

numpy version: 1.22.3
numpy version: 1.4.2
rdkit version: 2022.03.3


In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, GraphDescriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

Vamos começar com um exemplo simples, calculando alguns descritores para a molécula de etanol.

In [3]:
smi = "CCO"
mol = Chem.MolFromSmiles(smi)

Alguns descritores são valores **escalares** que representam uma molécula como um todo, como a sua massa molecular relativa. 

In [4]:
MolWt = Descriptors.MolWt(mol)
print("Massa molecular relativa:", MolWt)

Massa molecular relativa: 46.069


Também podemos usar **contagens** (*counts*) de propriedades, como número de átomos pesados ou de aceptores/doadores de hidrogênio. 

In [5]:
HeavyAtomCount = Descriptors.HeavyAtomCount(mol)
print("Contagem de átomos pesados:", HeavyAtomCount)

Contagem de átomos pesados: 3


In [6]:
NumHAcceptors = Descriptors.NumHAcceptors(mol)
print("Numero de aceptores de ligação de hidrogênio:", NumHAcceptors)

Numero de aceptores de ligação de hidrogênio: 1


In [7]:
NumHDonors = Descriptors.NumHDonors(mol)
print("Numero de doadores de ligação de hidrogênio:", NumHDonors)

Numero de doadores de ligação de hidrogênio: 1


Muitos descritores representam alguma **propriedade físico-química calculada**, como o coeficiente de partição água/octanol calculado (MolLogP) e a refratividade molar (MolMR). 

In [8]:
MolLogP = Descriptors.MolLogP(mol)
print("logP calculado:", MolLogP)

logP calculado: -0.0014000000000000123


In [9]:
MolMR = Descriptors.MolMR(mol)
print("Refratividade molar calculada:", MolMR)

Refratividade molar calculada: 12.759800000000002


Alguns descritores são independentes da estrutura molecular (como a massa molecular relativa). Outros, dependem da **conectividade** da molécula, ou seja, sua estrutura 2D. Alguns exemplos são MolLogP, índices topológicos (BertzCT, Hall-Kier Chi, Kappa) e eletrotopológicos (EStateIndex).

In [10]:
mol1 = Chem.MolFromSmiles("BrCCCC")
mol2 = Chem.MolFromSmiles("BrCC(C)C")

print("Dois isômeros estruturais")

print("Apresentam a mesma massa molecular relativa")
print(f"{Descriptors.MolWt(mol1):.3f}")
print(f"{Descriptors.MolWt(mol2):.3f}")

print("Mas diferentes valores de MolLogP")
print(f"{Descriptors.MolLogP(mol1):.3f}")
print(f"{Descriptors.MolLogP(mol2):.3f}")

print("E diferentes valores de BertzCT, um índice topológico relacionado à complexidade molecular")
print(f"{GraphDescriptors.BertzCT(mol1):.3f}")
print(f"{GraphDescriptors.BertzCT(mol2):.3f}")


Dois isômeros estruturais
Apresentam a mesma massa molecular relativa
137.020
137.020
Mas diferentes valores de MolLogP
2.181
2.037
E diferentes valores de BertzCT, um índice topológico relacionado à complexidade molecular
11.119
17.610


Alguns descritores são calculados a partir de uma **estrutura 3D**, por exemplo, os descritores WHIM. Para isso, precisamos primeiramente gerar uma estrutura 3D. No RDKit, podemos usar a função `AllChem.EmbedMolecule()`

In [11]:
from rdkit.Chem.rdMolDescriptors import CalcWHIM

In [12]:
mol = Chem.MolFromSmiles('OCCc1ccn2cnccc12')
mol = Chem.AddHs(mol)  # é uma boa ideia adicionar os átomos de hidrogênio antes de calcular a estrutura 3D

AllChem.EmbedMolecule(mol)

0

In [13]:
# Cálculo dos descritores WHIM - 114 descritores são calculados
whim_descs = CalcWHIM(mol)
print(len(whim_descs))

114


Há também descritores que **combinam** dois ou mais descritores. 

No RDKit, por exemplo, temos SlogP_VSA, SMR_VSA, PEOE_VSA e EState_VSA que envolvem o cálculo da contribuição de cada átomo na molécula para uma propriedade molecular (LogP, MR, carga parcial ou E-state, respectivamente) junto com a contribuição de cada átomo para uma medida aproximada da área de superfície molecular (a parte VSA dos descritores). Em seguida, atribuem-se os átomos a compartimentos (*bins*) com base nas contribuições para a propriedade e, então, somam-se as contribuições para a VSA para cada átomo em um compartimento. Veja mais detalhes neste [link](https://greglandrum.github.io/rdkit-blog/posts/2023-04-17-what-are-the-vsa-descriptors.html)

Note que, em geral, quanto mais abstratos os descritores, mais difícil será sua **interpretabilidade**, sendo, portanto, difícil utilizá-los como parâmetro para a proposição de novas estruturas químicas otimizadas. 

Concluímos reforçando que as possibilidades são muitas para calcular valores numéricos baseados em estruturas moleculares. Somente no RDKit são mais de 200 valores que podem ser gerados. Com o módulo [molfeat](https://molfeat.datamol.io/), por exemplo, podemos calcular mais de 1800 descritores 2D e 3D usando o pacote Mordred, além de diversos tipos de *fingerprints* e descritores baseados em redes neurais (por exemplo, ChemGPT, GIN e JTVAE).

## Calculando todos os descritores disponíveis no RDKit

Vou concluir mostrando esta função que recebe como entrada uma lista de SMILES e retorna um `pandas.DataFrame()` contendo todos os descritores disponíveis no RDKit.

O código está bem simples e não levei em conta fatores importantes, como a padronização das estruturas e a possibilidade de erros na conversão dos SMILES. Você pode modificar o código como achar necessário.

In [14]:
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import Descriptors, rdFingerprintGenerator
from rdkit.ML.Descriptors import MoleculeDescriptors


def descriptors_2D_from_smiles(smi_list):
    ms = [Chem.MolFromSmiles(smi) for smi in smi_list]
    # Descritores
    # Anotando todos os nomes dos descritores em uma lista
    names = [name[0] for name in Descriptors.descList]

    # Calculando os descritores e adicionando à lista "descs"
    calc = MoleculeDescriptors.MolecularDescriptorCalculator(names)
    descs = [calc.CalcDescriptors(m) for m in ms]
    # Obs: nas versões mais recentes (2024) do RDKit, "descs" pode ser calculado de uma forma mais simples:
    # descs = [Descriptors.CalcMolDescriptors(mol) for mol in ms]
    
    # Convertendo o resultado a um pd.DataFrame com os nomes dos descritores nas colunas
    descriptors_df = pd.DataFrame(descs, columns=names)
    return descriptors_df

Veja como aplicar a função para calcular os descritores para as estruturas no [conjunto de dados de Delaney (solubilidade)](https://www.kaggle.com/c/drug-solubility-challenge/data).

In [15]:
# Importando o conjunto de dados de Delaney como um pandas.DataFrame
df = pd.read_csv("../datasets/delaney-processed.csv")
# Mostrar as 10 primeiras linhas
df.head(10)

,Compound ID,ESOL predicted log solubility in mols per litre,Minimum Degree,Molecular Weight,Number of H-Bond Donors,Number of Rings,Number of Rotatable Bonds,Polar Surface Area,measured log solubility in mols per litre,smiles
0,Amigdalin,-0.974,1,457.432,7,3,7,202.32,-0.77,OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)...
1,Fenfuram,-2.885,1,201.225,1,2,2,42.24,-3.30,Cc1occc1C(=O)Nc2ccccc2
2,citral,-2.579,1,152.237,0,0,4,17.07,-2.06,CC(C)=CCCC(C)=CC(=O)
3,Picene,-6.618,2,278.354,0,5,0,0.00,-7.87,c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43
4,Thiophene,-2.232,2,84.143,0,1,0,0.00,-1.33,c1ccsc1
5,benzothiazole,-2.733,2,135.191,0,2,0,12.89,-1.50,c2ccc1scnc1c2
6,"2,2,4,6,6'-PCB",-6.545,1,326.437,0,2,1,0.00,-7.32,Clc1cc(Cl)c(c(Cl)c1)c2c(Cl)cccc2Cl
7,Estradiol,-4.138,1,272.388,2,4,0,40.46,-5.03,CC12CCC3C(CCc4cc(O)ccc34)C2CCC1O
8,Dieldrin,-4.533,1,380.913,0,5,0,12.53,-6.29,ClC4=C(Cl)C5(Cl)C3C1CC(C2OC12)C3C4(Cl)C5(Cl)Cl
9,Rotenone,-5.246,1,394.423,0,5,3,63.22,-4.42,COc5cc4OCC3Oc2c1CC(Oc1ccc2C(=O)C3c4cc5OC)C(C)=C


In [16]:
# Vamos manter somente a coluna contendo os SMILES e o valor da variável resposta
# (*measured log solubility in mols per litre*)
df = df[["smiles", "measured log solubility in mols per litre"]]

# Renomeando as colunas
df.columns=["SMILES", "Solubilidade_medida"]

In [17]:
# Agora basta calcular os descritores
descriptors_df = descriptors_2D_from_smiles(df["SMILES"])

# Vamos ver as cinco primeiras linhas
descriptors_df.head()

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,10.253329,-1.701605,10.253329,0.486602,0.217518,457.432,430.216,457.158411,178,0,...,0,0,0,0,0,0,0,0,0,0
1,11.724911,-0.145880,11.724911,0.145880,0.811283,201.225,190.137,201.078979,76,0,...,0,0,0,0,0,0,0,0,0,0
2,10.020498,0.845090,10.020498,0.845090,0.343706,152.237,136.109,152.120115,62,0,...,0,0,0,0,0,0,0,0,0,0
3,2.270278,1.301055,2.270278,1.301055,0.291526,278.354,264.242,278.109550,102,0,...,0,0,0,0,0,0,0,0,0,0
4,2.041667,1.712963,2.041667,1.712963,0.448927,84.143,80.111,84.003371,26,0,...,0,0,0,0,0,0,0,1,0,0


In [18]:
# Nesta versão do RDKit temos 208 descritores
descriptors_df.shape

(1128, 208)

In [19]:
# Verificar se houve alguma falha no cálculo dos descritores
descriptors_df.isna().sum().sum()

0

In [20]:
# Podemos adicionar os valores de solubilidade ao DataFrame
descriptors_df["Solubilidade_medida"] = df["Solubilidade_medida"]

In [21]:
# Salvar o DataFrame como .csv
descriptors_df.to_csv("../datasets/Delaney_descriptors.csv", sep=";", index=False)